In [ ]:
import numpy as np
from qutip import *
from matplotlib import pyplot as plt

In [ ]:
def print_quantum_state_as_superposition(state):
    """
    Helper function to print quantum state as superposition of basis functions
    """
    state = state.unit()
    dimension = int(np.log2(state.shape[0]))
    state_vector = state.full().flatten()
    basis_states = ['0', '1']
    
    superposition = []
    for i, amplitude in enumerate(state_vector):
        abs_amplitude = np.abs(amplitude)

        if round(abs_amplitude, 2) > 0:
            sign = '-' if amplitude.real < 0 else '+'
            binary_state = ''.join(basis_states[int(bit)] for bit in f"{i:0{dimension}b}")
            superposition.append(f"{sign} {abs_amplitude:.2f}|{binary_state}⟩")

    # Removing the first sign
    if superposition and superposition[0][0] == '+':
        superposition[0] = superposition[0][2:]

    superposition_str = ' '.join(superposition).replace('+-', '- ')
    
    print(f"|ψ⟩ = {superposition_str}")

# MEsolve efficient loop

In [ ]:
import numpy as np
from qutip import *
from matplotlib import pyplot as plt

In [ ]:
def bose_einstein_distribution(energy_diff, temperature):
    if temperature == 0:
        return 0
    return 1 / (np.exp(energy_diff / (temperature)) - 1)

def gab(omega1, omega2, lambda_2):
    if omega1 >= omega2:
        return 0
    return np.sqrt(lambda_2)

def c_ops_gen_new(t, args):
    H_inst_t = args['args_H'](t)  # Instantaneous Hamiltonian at time t
    eigenvalues, eigenstates = H_inst_t.eigenstates()

    c_ops = []
    for a_i, a in enumerate(eigenstates):

        omega_a = eigenvalues[a_i]
        for b_i, b in enumerate(eigenstates[a_i+1:]):
            omega_b = eigenvalues[a_i + b_i + 1]
            
            c = b*a.dag()
            c_coef = 0
            for i in range(args['n_qubits']):
                sigmam_i = tensor([sigmam() if j == i else I for j in range(args['n_qubits'])])
                
                if np.abs(omega_a - omega_b) < 1e-6:
                    continue  # Skip nearly degenerate states
                
                # For decay transitions (omega_b > omega_a)
                if omega_b > omega_a:
                    Nba = bose_einstein_distribution(omega_b - omega_a, args['T'])
                    g_ba = gab(omega_a, omega_b, args['lambda_2'])

                    ma_ba = a.dag() * sigmam_i * b * b.dag() * sigmam_i.dag() * a
#                     ma_ba = np.abs(ma_ba)
#                     if ma_ba > 1e-6:
                    decay_op = np.sqrt(Nba * (g_ba ** 2) * ma_ba)
                    c_coef += decay_op

                # For excitation transitions (omega_a > omega_b)
                elif omega_a > omega_b:
                    Nab = bose_einstein_distribution(omega_a - omega_b, args['T'])
                    g_ab = gab(omega_b, omega_a, args['lambda_2'])
                    ma_ab = b.dag() * sigmam_i * a * a.dag() * sigmam_i.dag() * b
                    excitation_op = np.sqrt((Nab + 1) * (g_ab ** 2) * ma_ab)                    
                    c_coef += excitation_op
                    
                    
            if c_coef > 1e-5:
                c_ops.append(c.to('csr')*c_coef)

    return c_ops

In [ ]:
# n_qubits = 4
sigma = 1
omega_max = 10
max_time = 10
times = np.linspace(0, max_time,301) # has to be +1 for the final time stamp

nsteps = 1e5
gamma_sqrt = 0.1
T = 40 # temp

# Time-dependent functions
omega_t = lambda t: omega_max * np.exp(-0.5 * ((t - max_time/2) / sigma) ** 2)    
delta_t = lambda t: 2 * omega_max * (t / (max_time / 2) - 1)

Eps = [0, 5, 10, 25, 50, 100]

rho0 = logical_zero * logical_zero.dag()
rho1 = logical_one * logical_one.dag()
rho_A_0 = ancilla_zero * ancilla_zero.dag()
rho_A_1 = ancilla_one * ancilla_one.dag()
e_ops = [rho0, rho1]

plt.figure(figsize=(8, 6))

# Plotting Ideal
def H_se(t):
    return omega_t(t) * X_L + delta_t(t) * Z_L

results_se = sesolve(H_se, logical_zero, times, e_ops=e_ops)
# plt.plot(times, results_se.expect[0], linestyle="--", label=r"$\rho_0$ - ideal")
plt.plot(times, results_se.expect[1], linestyle="--", label=r"ideal")

# Loop over different values of Ep
for Ep in Eps:

    print(Ep)
    
    rho_init = initial_state * initial_state.dag()
    # e_ops = [rho0, rho1, rho_A_0, rho_A_1]
    e_ops = [rho0, rho1]
    
    # Time-dependent Hamiltonian
    def H_t(t):
        # H_RAP = X_L_1 * omega_t(t) + Z_L_1 * delta_t(t)
        H_RAP = X_L * omega_t(t) + Z_L * delta_t(t)
        
        # Ht = H_RAP + Ep * H_supp(x, z, nu1, nu2, lambda1, lambda2, opt)
        Ht = H_RAP - Ep * sum(G)
#       Z_A = tensor(Z, I, Z, I)
#       Ht = H_RAP - Ep * (Z_A)
        return Ht.to('csr')  # Assuming H_x and H_z are valid Qobjs

    args_C = {"args_H": H_t,
              "n_qubits": n_qubits,
              "lambda_2":gamma_sqrt,
              "T":T}

    prev_time = times[0]
    expects_0 = []
    expects_1 = []
    expects_0_A = []
    expects_1_A = []
    
    time_inter = times[1] - times[0]

    tot_time = times[1:]
    tot_time = np.append(tot_time, times[-1] + time_inter)


    # Time evolution in steps
    for dt in tot_time:
        c_ops = c_ops_gen_new(dt, args=args_C)
        t_steps = [prev_time, dt]
        results = mesolve(H_t, rho_init, t_steps, c_ops=c_ops, args=args_C, e_ops=e_ops, 
                          options={'store_states': True})

        # Updating
        prev_time = dt
        rho_init = results.states[-1]

        # Saving
        expects_1.append(results.expect[1][0])
        expects_0.append(results.expect[0][0])
    
    expects_all = [expects_1[i] for i in range(len(expects_1))]
    
    # Plotting results
    plt.plot(times, expects_all, label=r"$E_{P}$=" + str(Ep))

    
    
plt.legend()
plt.xlabel('[a.u.]')
plt.ylabel('Fidelity')
plt.show()

# Evolve state master equation in time loop

In [ ]:
def op_on_qubit(op, i, N=3):
    return tensor([op if k == i else I for k in range(N)])

In [ ]:
# Pauli operators on 3 qubits
X1, X2, X3 = [op_on_qubit(X, i) for i in range(3)]
Z1, Z2, Z3 = [op_on_qubit(Z, i) for i in range(3)]

# Stabilizers
S1 = Z1 * Z2
S2 = Z2 * Z3

# Collapse operators (bit-flip noise)
c_ops = [np.sqrt(gamma) * i for i in [X1, X2, X3]]

# Initial state: logical |0_L> = |000>
psi0 = tensor(basis(2,0), basis(2,0), basis(2,0))
rho = ket2dm(psi0)

# Target logical states for fidelity
zero = tensor(basis(2,0), basis(2,0), basis(2,0))
one  = tensor(basis(2,1), basis(2,1), basis(2,1))

# Initialize filtered stabilizer signals
I1_bar, I2_bar = 0.0, 0.0

# Record fidelity
fidelities_0 = []
fidelities_1 = []

errors = []


I1s = []
I2s = []

# Time evolution loop
for t in tlist:
    # Hamiltonian            
    H = (X1 * X2 * X3)*omega_t(t) + ((Z1 + Z2 + Z3)/3)*delta_t(t)

    # Effective SME step
    dW1, dW2 = np.random.normal(scale=np.sqrt(dt), size=2)

    meas1 = np.sqrt(2 * gamma_m * eta) * (S1 * rho + rho * S1 - 2 * expect(S1, rho) * rho)
    meas2 = np.sqrt(2 * gamma_m * eta) * (S2 * rho + rho * S2 - 2 * expect(S2, rho) * rho)

    drho = -1j * (H * rho - rho * H)
    for c in c_ops:
        drho += c * rho * c.dag() - 0.5 * (c.dag() * c * rho + rho * c.dag() * c)
        
    drho += gamma_m * (S1 * rho * S1 - 0.5 * (S1**2 * rho + rho * S1**2))
    drho += gamma_m * (S2 * rho * S2 - 0.5 * (S2**2 * rho + rho * S2**2))
    
    drho += meas1 * dW1 + meas2 * dW2

    rho = (rho + dt * drho).unit()

    # Measurement currents (with noise)
    I1 = expect(S1, rho) + dW1 / np.sqrt(dt)
    I2 = expect(S2, rho) + dW2 / np.sqrt(dt)
    

    # Exponential filtering
    I1_bar = (1 - alpha) * I1_bar + alpha * I1
    I2_bar = (1 - alpha) * I2_bar + alpha * I2
    
    I1s.append(I1_bar)
    I2s.append(I2_bar)
    
    # Some Condition inside the propagation
    # # Double threshold logic
    # if I1_bar < theta1 and I2_bar > theta2:
    #     rho = X1 * rho * X1
    #     I1_bar, I2_bar = 1, 1
    #     errors.append(("X1", t))
    # elif I1_bar < theta1 and I2_bar < theta1:
    #     rho = X2 * rho * X2
    #     I1_bar, I2_bar = 1, 1
    #     errors.append(("X2", t))
    # elif I1_bar > theta2 and I2_bar < theta1:
    #     rho = X3 * rho * X3
    #     I1_bar, I2_bar = 1, 1
    #     errors.append(("X3", t))

    
    # fidelities.append((psi_target.dag() * rho * psi_target))
    fidelities_0.append((zero.dag() * rho * zero))
    fidelities_1.append((one.dag() * rho * one))